In [ ]:
import numpy as np

def exp_map(omega_hat):
    theta = np.linalg.norm([omega_hat[2, 1], omega_hat[0, 2], omega_hat[1, 0]])
    if theta < 1e-10:
        return np.eye(3)
    
    A = omega_hat / theta
    return np.eye(3) + np.sin(theta) * A + (1 - np.cos(theta)) * np.dot(A, A)


def hat(v):
    return np.array([[0, -v[2], v[1]],
                     [v[2], 0, -v[0]],
                     [-v[1], v[0], 0]])

def derivatives(m, R, *args):
    g, I_inv, m_b, rho, e_3 = args
    omega = I_inv @ m
    m_dot = np.cross(m, omega) + m_b * g * np.cross(rho, R.T @ e_3)
    return m_dot, omega


def RK4_step(f, m, R, h, *args):
    k1_m, k1_w = f(m, R, *args)
    k2_m, k2_w = f(m + 0.5 * h * k1_m, R @ exp_map(0.5 * h * k1_w), *args)
    k3_m, k3_w = f(m + 0.5 * h * k2_m, R @ exp_map(0.5 * h * k2_w), *args)
    k4_m, k4_w = f(m + h * k3_m, R @ exp_map(h * k3_w), *args)

    m += h / 6 * (k1_m + 2 * k2_m + 2 * k3_m + k4_m)
    R = R @ exp_map(h / 6 * (k1_w + 2 * k2_w + 2 * k3_w + k4_w))

    return m, R

def derivatives(m, R, *args):
    g, I_inv, m_b, rho, e_3 = args
    omega = I_inv @ m
    m_dot = np.cross(m, omega) + m_b * g * np.cross(rho, R.T @ e_3)
    omega_hat = hat(omega)  
    R_dot = R @ omega_hat
    return m_dot, R_dot

def RK4_step(f, m, R, h, *args):
    k1_m, k1_R_dot = f(m, R, *args)
    k2_m, k2_R_dot = f(m + 0.5 * h * k1_m, R + 0.5 * h * k1_R_dot, *args)
    k3_m, k3_R_dot = f(m + 0.5 * h * k2_m, R + 0.5 * h * k2_R_dot, *args)
    k4_m, k4_R_dot = f(m + h * k3_m, R + h * k3_R_dot, *args)

    m += h / 6 * (k1_m + 2 * k2_m + 2 * k3_m + k4_m)
    R += h / 6 * (k1_R_dot + 2 * k2_R_dot + 2 * k3_R_dot + k4_R_dot)

    return m, R


def simulate(m, R, m_b, g, rho, e_3, T, I):
    h = T[1] - T[0]
    lst_m = np.zeros((len(T), 3))
    lst_R = np.zeros((len(T), 3, 3))
    I_inv = np.linalg.inv(I)
    args = (g, I_inv, m_b, rho, e_3)
    for i, t in enumerate(T):
        m, R = RK4_step(derivatives, m, R, h, *args)
        lst_m[i] = m
        lst_R[i] = R
    return lst_m, lst_R


In [ ]:
# def calculate_intertia(mass_ball, radius_ball, length_rod):
#     assert mass_ball > 0
#     assert 0 < radius_ball < length_rod
#     I_ball_center = (2/5) * mass_ball * radius_ball**2

#     # Using the parallel axis theorem to find the moment of inertia about the pivot 
#     I_pendulum = I_ball_center + mass_ball * (radius_ball/2)**2 + mass_ball * length_rod**2

#     # Inertia tensor at the pivot
#     I_pendulum_matrix = np.diag([I_pendulum - I_ball_center, I_pendulum - I_ball_center, I_ball_center])
#     return I_pendulum_matrix

def calculate_intertia(mass_ball, radius_ball, length_rod):
    I = mass_ball * length_rod **2
    return np.diag([I, I, I])

e_3 = np.array([0, 0, 1])
mass_ball = 5.0
g = 9.81
rho = np.array([0, 0, 1])
I = calculate_intertia(mass_ball = mass_ball, 
                       radius_ball = 0.2, 
                       length_rod = 1.0)


# The initial state
# m = np.array([1.5, 2.5, 4.5])
m = np.array([7.5, 2.5, 4.5])
R = np.eye(3)       

T = np.linspace(0, 10, 1000)
_, states_R = simulate(m, R, mass_ball, g, rho, e_3, T, I)

In [ ]:
for i, R in enumerate(states_R): 
    # Check that R is a rotation matrix
    assert np.allclose(np.linalg.det(R), 1), f"the determinant of R_{i} is {np.linalg.det(R)}"
    assert np.allclose(R @ R.T, np.eye(3)), f"R @ R.T is {R @ R.T}"
    assert np.allclose(R.T @ R, np.eye(3)), f"R.T @ R is {R.T @ R}"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define the radius of the hemisphere
radius = 1.0
start_loc = np.array([0, -radius, -radius])
start_loc = start_loc / np.linalg.norm(start_loc) 

rotated_endpoints = np.array([R @ start_loc for R in states_R])
rotated_endpoints = rotated_endpoints[:600]


############# Plotting the pendulum motion #############
# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Create a meshgrid for the x, y values
x = np.linspace(-radius, radius, 100)
y = np.linspace(-radius, radius, 100)
x, y = np.meshgrid(x, y)

# Define z as a function of x, y to create a hemisphere shape
# Only plot where the sphere's equation holds and z is non-negative (downward facing hemisphere)
with np.errstate(invalid='ignore'):
    z = np.sqrt(radius**2 - x**2 - y**2)
z[np.isnan(z)] = 0  # Handle the domain issues outside the hemisphere

# Plot the hemisphere
ax.plot_surface(x, y, -z, color='c', alpha=0.3)  # negative z for downward facing

# Plot the original stick endpoint at [0, 0, radius]
ax.scatter(start_loc[0], start_loc[1], start_loc[2], color='b', label='Original Stick End')

# Plot the rotated stick endpoints
ax.plot(rotated_endpoints[:, 0], rotated_endpoints[:, 1], rotated_endpoints[:, 2], color='r', label='Pendulum Endpoints')

# Set plot labels and legend
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.legend()

# Set equal aspect ratio for all axes
ax.set_box_aspect([1,1,1])  # Equal aspect ratio

# Show the plot
plt.show()


In [ ]:
for R in rotated_endpoints:
    assert np.allclose(np.linalg.norm(R), 1), f"the norm of R is {np.linalg.norm(R)}"